In [6]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from datetime import datetime
import os
load_dotenv(override=True)

True

In [3]:
from profiles import ContentAccount
from profiles_client import read_content_account, read_content_strategy
from trends import get_top_trending_topics

In [5]:
# 1. Create or load a profile
profile = ContentAccount.get("AI_Content_Curator")

In [ ]:
# 2. Reset profile with a strategy
profile.reset("Focus on top trending AI breakthroughs with high engagement potential")

In [7]:
# 3. Check top AI trends right now
top_trends = get_top_trending_topics(3)  # Top 3 topics
print("Top AI Trends Today:")
for t in top_trends:
    print(f"- {t.topic} (Score: {t.score:.2f})")

Error fetching Twitter trends: 429 Too Many Requests
Too Many Requests
Top AI Trends Today:
- GPT (Score: 100.00)
- ChatGPT (Score: 86.66)
- AGI (Score: 66.54)


In [8]:
# 4. Create content for the first trending topic
if top_trends:
    first_topic = top_trends[0].topic
    profile.create_content(
        topic=first_topic,
        platform="twitter",
        content_type="thread",
        rationale=f"High score {top_trends[0].score:.1f} — good for quick engagement"
    )

# 5. Show profile report
print("\nProfile Report:")
print(profile.report())

Used 1 credits. Remaining: 99

Profile Report:
{
  "name": "ai_content_curator",
  "credits": 99,
  "strategy": "Focus on top trending AI breakthroughs with high engagement potential",
  "content_history": [
    {
      "topic": "GPT",
      "platform": "twitter",
      "content_type": "thread",
      "trend_score": 100.0,
      "timestamp": "2025-08-09 16:56:37",
      "strategy_rationale": "High score 100.0 \u2014 good for quick engagement",
      "engagement_score": 1200.0
    }
  ],
  "engagement_time_series": [
    [
      "2025-08-09 16:56:37",
      1200.0
    ],
    [
      "2025-08-09 16:56:37",
      1200.0
    ]
  ],
  "platform_stats": {
    "blog": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "twitter": {
      "posts": 1,
      "total_engagement": 1200.0
    },
    "linkedin": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "newsletter": {
      "posts": 0,
      "total_engagement": 0.0
    }
  },
  "topic_coverage": {
    "GPT": 1
  },
  "tot

In [9]:
# 6. Show recent content history
print("\nRecent Content History:")
for c in profile.get_content_history():
    print(c)


Recent Content History:
{'topic': 'GPT', 'platform': 'twitter', 'content_type': 'thread', 'trend_score': 100.0, 'timestamp': '2025-08-09 16:56:37', 'strategy_rationale': 'High score 100.0 — good for quick engagement', 'engagement_score': 1200.0}


# Use MCP server 

In [9]:
# Now let's use our accounts server as an MCP server

params = {"command": "python", "args": ["profiles_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='create_content', description='Create content for a topic on a platform', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}, 'topic': {'title': 'Topic', 'type': 'string'}, 'platform': {'title': 'Platform', 'type': 'string'}, 'content_type': {'title': 'Content Type', 'type': 'string'}, 'rationale': {'title': 'Rationale', 'type': 'string'}}, 'required': ['name', 'topic', 'platform', 'content_type', 'rationale'], 'title': 'create_contentArguments', 'type': 'object'}, annotations=None),
 Tool(name='skip_content', description='Skip creating content for a topic', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}, 'topic': {'title': 'Topic', 'type': 'string'}, 'rationale': {'title': 'Rationale', 'type': 'string'}}, 'required': ['name', 'topic', 'rationale'], 'title': 'skip_contentArguments', 'type': 'object'}, annotations=None),
 Tool(name='promote_content', description='Promote existing content to a new platform', inputSchema={'propertie

In [ ]:
# Use MCP server with SDK 
instructions = (
            "You are a content account manager AI. "
            "You can use MCP tools to check content account reports, "
            "create content, view performance, and get top trends."
        )
request = (
            "My profile name is AI_Content_Curator. "
            "What is my recent content performance?"
        )

In [6]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(
            name="content_manager",
            instructions=instructions,
            model="gpt-4o-mini",
            mcp_servers=[mcp_server]
        )
    with trace("content_manager"):
            result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your recent content performance as **AI_Content_Curator** is as follows:

- **Total Content Created:** 1
- **Total Engagement:** 1,200
- **Average Engagement per Post:** 1,200

### Platform Analysis:
- **Twitter:**
  - Posts: 1
  - Total Engagement: 1,200
  - Average Engagement per Post: 1,200

### Topic Performance:
- **GPT:**
  - Content Count: 1
  - Total Engagement: 1,200
  - Average Trend Score: 100
  - Average Engagement: 1,200

### Credits Remaining: 
- 99 credits

# Now let's build our own MCP Client for Resources

In [7]:
ContentAccount.get("AI_Content_Curator").report()

'{\n  "name": "ai_content_curator",\n  "credits": 99.0,\n  "strategy": "Focus on top trending AI breakthroughs with high engagement potential",\n  "content_history": [\n    {\n      "topic": "GPT",\n      "platform": "twitter",\n      "content_type": "thread",\n      "trend_score": 100.0,\n      "timestamp": "2025-08-09 16:56:37",\n      "strategy_rationale": "High score 100.0 \\u2014 good for quick engagement",\n      "engagement_score": 1200.0\n    }\n  ],\n  "engagement_time_series": [\n    [\n      "2025-08-09 16:56:37",\n      1200.0\n    ],\n    [\n      "2025-08-09 16:56:37",\n      1200.0\n    ],\n    [\n      "2025-08-09 21:58:31",\n      1200.0\n    ]\n  ],\n  "platform_stats": {\n    "blog": {\n      "posts": 0,\n      "total_engagement": 0.0\n    },\n    "twitter": {\n      "posts": 1,\n      "total_engagement": 1200.0\n    },\n    "linkedin": {\n      "posts": 0,\n      "total_engagement": 0.0\n    },\n    "newsletter": {\n      "posts": 0,\n      "total_engagement": 0.0\n

In [1]:
from profiles_client import get_profiles_tools_openai, read_content_account, list_profiles_tools

In [4]:
resource = await read_content_account("AI_Content_Curator")
print(resource)

{
  "name": "ai_content_curator",
  "credits": 99.0,
  "strategy": "Focus on top trending AI breakthroughs with high engagement potential",
  "content_history": [
    {
      "topic": "GPT",
      "platform": "twitter",
      "content_type": "thread",
      "trend_score": 100.0,
      "timestamp": "2025-08-09 16:56:37",
      "strategy_rationale": "High score 100.0 \u2014 good for quick engagement",
      "engagement_score": 1200.0
    }
  ],
  "engagement_time_series": [
    [
      "2025-08-09 16:56:37",
      1200.0
    ],
    [
      "2025-08-09 16:56:37",
      1200.0
    ],
    [
      "2025-08-09 21:58:31",
      1200.0
    ],
    [
      "2025-08-13 10:42:36",
      1200.0
    ]
  ],
  "platform_stats": {
    "blog": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "twitter": {
      "posts": 1,
      "total_engagement": 1200.0
    },
    "linkedin": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "newsletter": {
      "posts": 0,
      "total_engagemen

# Experiment with more MCP servers

### 1. Create the MCPServerStdio for each MCP params

In [8]:
# **Let's start by gathering the MCP params for our content curator**

# API Keys for trend data sources
reddit_client_id = os.getenv("REDDIT_CLIENT_ID")
reddit_client_secret = os.getenv("REDDIT_CLIENT_SECRET")
twitter_bearer_token = os.getenv("TWITTER_BEARER_TOKEN")
youtube_api_key = os.getenv("YOUTUBE_API_KEY")
brave_api_key = os.getenv("BRAVE_API_KEY")

In [14]:
# **Now set up the MCP server parameters**

# Content management MCP (always use local server)
content_accounts_mcp = {"command": "python", "args": ["profiles_server.py"]}

# Trends MCP - use real APIs if available
trends_mcp = {"command": "python", "args": ["trends_server.py"]}


# Content publishing MCP (local server for now, can be extended to real APIs)
content_publishing_mcp = {"command": "python", "args": ["content_server.py"]}

# send email 
resend_mcp = {"command": "python", "args": ["resend_server.py"]}

curator_mcp_server_params = [
    content_accounts_mcp,
    trends_mcp,
    content_publishing_mcp,
    resend_mcp,
]



In [11]:
# AI researcher MCP params
brave_env = {"BRAVE_API_KEY": brave_api_key}

ai_researcher_mcp_server_params = {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": brave_env,
}

async with MCPServerStdio(params=ai_researcher_mcp_server_params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [12]:
ai_researcher_mcp_server_params = [
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env},
]

In [15]:
### Now create the MCPServerStdio for each
ai_researcher_mcp_servers = [
    MCPServerStdio(params, client_session_timeout_seconds=120) 
    for params in ai_researcher_mcp_server_params
]

curator_mcp_servers = [
    MCPServerStdio(params, client_session_timeout_seconds=120) 
    for params in curator_mcp_server_params
]

all_mcp_servers = curator_mcp_servers + ai_researcher_mcp_servers

### 2: Make AI Trends Researcher Agent and turn it into a tool

In [16]:
async def get_ai_researcher(mcp_servers) -> Agent:
    instructions = f"""You are an AI trends researcher. You search the web for the latest AI developments,
breakthroughs, tool launches, research papers, and industry news that would be excellent for content creation.

Focus on finding:
- Trending AI topics and developments
- New AI tools and applications
- Research breakthroughs and papers
- Industry news and company announcements
- AI policy and regulatory developments

Based on the request, carry out comprehensive research and respond with:
1. Key trending topics with context
2. Content opportunities and suggested angles  
3. Platform recommendations (blog, Twitter, LinkedIn, newsletter)
4. Trend scores and engagement potential

Take time to make multiple searches to get a comprehensive overview.
If there isn't a specific request, find the most trending AI topics today.

Current datetime: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    
    researcher = Agent(
        name="AI_Trends_Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [17]:
# Use researcher as tool for other agents
async def get_ai_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_ai_researcher(mcp_servers)
    return researcher.as_tool(
        tool_name="AI_Trends_Researcher",
        tool_description="This tool researches trending AI topics and content opportunities online, \
            either based on your specific request for certain AI areas/companies, \
            or generally for the hottest AI news and trends. \
            Describe what kind of AI content research you're looking for."
    )

### 3: Test calling AI researcher directly to check it works

In [26]:

research_question = "What are the most trending AI developments and breakthroughs this week that would make great content?"
  
for server in ai_researcher_mcp_servers:
    await server.connect()

researcher = await get_ai_researcher(ai_researcher_mcp_servers)
    
with trace("AI_Researcher_Direct_Test"):
    result = await Runner.run(researcher, research_question, max_turns=10)
display(Markdown(result.final_output))


Here are the most trending AI developments and breakthroughs from August 2025 that could generate compelling content:

### Key Trending Topics

1. **Carnegie Mellon University AI Institute**:
   - **Context**: Launch of a new NSF-funded institute focused on leveraging AI for mathematical discovery. It aims to create models that can conjecture, prove, and visualize complex mathematical theorems.
   - **Content Angle**: Discuss AI's role in enhancing scientific research, particularly in mathematics. Explore potential implications for education and research methodologies.
  
2. **DeepSeek's Advancements**:
   - **Context**: Following a breakthrough, China prioritizes real-world applications over cutting-edge developments in AI.
   - **Content Angle**: Analyze how geopolitical dynamics affect AI development and deployment, with a focus on China's commitment to practical applications.

3. **AI-Assisted Brain Age Estimation**:
   - **Context**: A new study reveals that AI can accurately estimate brain age from MRI data, aiding in early diagnosis of neurological conditions.
   - **Content Angle**: Explore the intersection of AI and healthcare, highlighting the importance of early diagnosis in treatment strategies.

4. **Wan2.2 AI Video Generation**:
   - **Context**: Alibaba releases an open-source update (Wan2.2) for AI-driven video generation, improving generalizability and quality.
   - **Content Angle**: Discuss the implications of open-source AI tools for content creation and the future of media production.

5. **EU AI Act Implementation**:
   - **Context**: The upcoming enforcement of Europe’s first comprehensive AI regulatory framework, the EU AI Act, scheduled for August.
   - **Content Angle**: Delve into the challenges and opportunities presented by AI regulation, including potential impacts on tech companies and innovation.

### Content Opportunities & Suggested Angles

- **Interviews**: Conduct interviews with experts in AI ethics and regulation to illuminate the implications of the EU AI Act.
- **Case Studies**: Highlight successful case studies on the application of AI in mathematics and healthcare.
- **How-To Guides**: Create guides on utilizing open-source AI tools like Wan2.2 for video content creation.
- **Webinars**: Host discussions focused on the geopolitical impacts of AI development, especially in the context of China and the U.S.

### Platform Recommendations

- **Blog**: In-depth articles exploring case studies and theoretical implications.
- **Twitter**: Quick updates and engagement on developing stories, such as regulatory changes or new AI applications.
- **LinkedIn**: Professional insights discussing the impact of these advancements on industry sectors.
- **Newsletter**: A dedicated AI newsletter summarizing key breakthroughs and trends for ongoing engagement with subscribers.

### Trend Scores & Engagement Potential

- **Carnegie Mellon Institute Launch**: 8/10 - High engagement potential among academia and tech communities.
- **DeepSeek's Advancements**: 7/10 - Moderate engagement, suitable for discussions on geopolitical tech implications.
- **Brain Age AI**: 9/10 - High engagement among healthcare professionals and researchers.
- **Wan2.2 Release**: 8/10 - Strong interest from content creators and media producers, especially in the open-source community.
- **EU AI Act**: 9/10 - Critical topic offering substantial engagement potential across policy makers and business leaders.

These insights serve as a robust foundation for generating rich, engaging content about the latest developments in AI.

##### always look at trace
https://platform.openai.com/traces

### 4: Setting Up Content Curator Strategies

In [18]:
from profiles import ContentAccount

alex_strategy = "Focus on AI breakthroughs and cutting-edge research. Create technical deep-dives and analysis for developers and AI researchers. Prioritize long-form blog content and detailed LinkedIn posts."

sam_strategy = "Cover practical AI tools and applications that help businesses and individuals. Create beginner-friendly content, tutorials, and how-to guides. Focus on Twitter threads and accessible newsletter content."

timi_strategy = "Explore AI ethics, policy, and societal implications. Create thought-provoking content about responsible AI development and governance. Balance content across all platforms with emphasis on professional insights."


In [19]:
def setup_curator_accounts():
    """Set up curator accounts with strategies"""
    strategies = {
        "Alex": alex_strategy,
        "Sam": sam_strategy, 
        "Timi": timi_strategy
    }
    
    for name, strategy in strategies.items():
        account = ContentAccount.get(name)
        account.reset(strategy)
    
    return strategies

curator_strategies = setup_curator_accounts()

In [14]:
display(Markdown(await read_content_account("Timi")))

{
  "name": "timi",
  "credits": 100.0,
  "strategy": "Explore AI ethics, policy, and societal implications. Create thought-provoking content about responsible AI development and governance. Balance content across all platforms with emphasis on professional insights.",
  "content_history": [],
  "engagement_time_series": [
    [
      "2025-08-11 12:55:15",
      0.0
    ],
    [
      "2025-08-11 12:55:39",
      0
    ]
  ],
  "platform_stats": {
    "blog": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "twitter": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "linkedin": {
      "posts": 0,
      "total_engagement": 0.0
    },
    "newsletter": {
      "posts": 0,
      "total_engagement": 0.0
    }
  },
  "topic_coverage": {},
  "total_engagement": 0,
  "engagement_rate": 0.0,
  "platform_performance": {
    "blog": 0.0,
    "twitter": 0.0,
    "linkedin": 0.0,
    "newsletter": 0.0
  },
  "top_topics": [],
  "recent_content_count": 0
}

### 5: Create Content Curator Agent with resources in prompt

In [20]:
# Read resources into prompt
agent_name = "Timi"
account_details = await read_content_account(agent_name)
strategy = await read_content_strategy(agent_name)

instructions = f"""
You are an AI trends content curator named {agent_name}. Your account is under your name, {agent_name}.
You create engaging content about AI trends across multiple platforms (blog, Twitter, LinkedIn, newsletter).

Your content strategy:
{strategy}

Your current account status:
{account_details}

You have access to tools that allow you to research trending AI topics, get trend scores, and create content.
You have tools to buy/create content, skip content opportunities, and promote existing content.
You have tools to track performance and analyze your content portfolio.

Please use these tools to manage your content strategy. Research trends, evaluate opportunities, 
create content as you see fit, and don't wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to research trending AI topics and create compelling content.
Follow this workflow:
1. Research the latest trending AI developments and opportunities
2. Evaluate which topics align best with your strategy and have high engagement potential  
3. Create content on the most promising topics across appropriate platforms
4. Respond with a summary of your content creation activities and decisions
"""

### 6: Run the Content Curator

In [ ]:
for server in all_mcp_servers:
    await server.connect()
researcher_tool = await get_ai_researcher_tool(ai_researcher_mcp_servers)
    
curator = Agent(
        name=agent_name,
        instructions=instructions,
        tools=[researcher_tool],
        mcp_servers=curator_mcp_servers,
        model="gpt-4o-mini",
    )
with trace(f"Content_Curator_{agent_name}"):
    result = await Runner.run(curator, prompt, max_turns=30)
display(Markdown(result.final_output))

In [15]:
# And let's look at the results of the trading

await read_content_account(agent_name)

'{\n  "name": "timi",\n  "credits": 96.0,\n  "strategy": "Explore AI ethics, policy, and societal implications. Create thought-provoking content about responsible AI development and governance. Balance content across all platforms with emphasis on professional insights.",\n  "content_history": [\n    {\n      "topic": "Deep Dive into AI Transparency",\n      "platform": "blog",\n      "content_type": "article",\n      "trend_score": 0.0,\n      "timestamp": "2025-08-11 14:51:38",\n      "strategy_rationale": "This topic addresses the urgent need for transparency in AI systems, appealing to professionals and stakeholders interested in ethical implications.",\n      "engagement_score": 0.0\n    },\n    {\n      "topic": "Future of Work with AI",\n      "platform": "LinkedIn",\n      "content_type": "post",\n      "trend_score": 0.0,\n      "timestamp": "2025-08-11 14:51:38",\n      "strategy_rationale": "A timely examination of how AI is reshaping job markets, garners interest from profe

# Build our Agent

In [25]:
from curator import ContentCurator

GOOGLE_API_KEY: SET
GROQ_API_KEY: SET


In [ ]:
curator = ContentCurator("Timi")
await curator.run()

[2025-08-11 20:31:18,191] DEBUG - Creating trace Timi-content_creation with id trace_timi0mnxeaoonwpiwfzefkx2jdkpi80r
[2025-08-11 20:31:18,235] DEBUG - Setting current trace: trace_timi0mnxeaoonwpiwfzefkx2jdkpi80r


[DEBUG] get_model called with: gpt-4o-mini
[DEBUG] get_model called with: gpt-4o-mini


[2025-08-11 20:32:01,459] DEBUG - Creating span <agents.tracing.span_data.MCPListToolsSpanData object at 0x7f32585125d0> with id None
[2025-08-11 20:32:02,581] DEBUG - Creating span <agents.tracing.span_data.MCPListToolsSpanData object at 0x7f3258582bd0> with id None
[2025-08-11 20:32:02,990] DEBUG - Creating span <agents.tracing.span_data.MCPListToolsSpanData object at 0x7f32586c2d90> with id None
[2025-08-11 20:32:03,114] DEBUG - Creating span <agents.tracing.span_data.MCPListToolsSpanData object at 0x7f326833aa90> with id None
[2025-08-11 20:32:03,380] DEBUG - Creating span <agents.tracing.span_data.AgentSpanData object at 0x7f3258567f70> with id None
[2025-08-11 20:32:03,382] DEBUG - Running agent Timi (turn 1)
[2025-08-11 20:32:03,398] DEBUG - Creating span <agents.tracing.span_data.GenerationSpanData object at 0x7f3258542b10> with id None
[2025-08-11 20:32:03,421] DEBUG - [
  {
    "content": "You are Timi, an AI trends content curator and creator.\nYour mission is to identify tr

In [31]:
await read_content_account("Timi")

'{\n  "name": "timi",\n  "credits": 92.0,\n  "strategy": "Explore AI ethics, policy, and societal implications. Create thought-provoking content about responsible AI development and governance. Balance content across all platforms with emphasis on professional insights.",\n  "content_history": [\n    {\n      "topic": "Deep Dive into AI Transparency",\n      "platform": "blog",\n      "content_type": "article",\n      "trend_score": 0.0,\n      "timestamp": "2025-08-11 14:51:38",\n      "strategy_rationale": "This topic addresses the urgent need for transparency in AI systems, appealing to professionals and stakeholders interested in ethical implications.",\n      "engagement_score": 0.0\n    },\n    {\n      "topic": "Future of Work with AI",\n      "platform": "LinkedIn",\n      "content_type": "post",\n      "trend_score": 0.0,\n      "timestamp": "2025-08-11 14:51:38",\n      "strategy_rationale": "A timely examination of how AI is reshaping job markets, garners interest from profe

In [32]:
from mcp_servers import curator_mcp_server_params, researcher_mcp_server_params

all_params = curator_mcp_server_params + researcher_mcp_server_params("Timi")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 5 MCP servers, and 32 tools
